In [18]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn import datasets
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('../data/balanced_sclaer_dataset_diabetes.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.isnull().sum().sum()

0

In [3]:
y = df["Diabetes_012"]
X = df.drop(["Diabetes_012"], axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [10]:
#KNN
knn_optimal = GridSearchCV(KNeighborsClassifier(), 
                                             {'n_neighbors': np.arange(3, 5, 1),
                                              'p': np.arange(1, 2, 1),
                                              "metric" : ["euclidean", "minkowski"]
                                              },
                                              ).fit(X_train[0:50000], y_train[0:50000])
knn_optimal.best_params_

{'metric': 'minkowski', 'n_neighbors': 3, 'p': 1}

In [20]:
#NB
gnb = GaussianNB()
GNB = gnb.fit(X_train[0:50000], y_train[0:50000])

In [19]:
#LB
log_reg_grid = {"C" : np.logspace(-4, 4, 20),
               "solver":["liblinear"]}
random_logr = RandomizedSearchCV(LogisticRegression(),
                               param_distributions=log_reg_grid,
                               cv=5,
                               n_iter=10,
                               random_state=0).fit(X_train[0:50000], y_train[0:50000])

In [25]:
random_logr.best_params_

{'solver': 'liblinear', 'C': 0.23357214690901212}

In [61]:
estimators = [[('model1', LogisticRegression(solver= 'liblinear', C = 0.23357214690901212))], 
[('model2',KNeighborsClassifier(metric= 'minkowski', n_neighbors = 3, p= 1))],
[('model3', GaussianNB())]]

In [42]:
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(solver= 'liblinear', C = 0.23357214690901212))

In [43]:
clf.fit(X_train, y_train).score(X_test, y_test)

0.8130894378436904

In [60]:
estimators[0]

[('model1', LogisticRegression(C=0.23357214690901212, solver='liblinear'))]

In [ ]:
# 'estimators': [LogisticRegression(solver= 'liblinear', C = 0.23357214690901212), 
#                                                              KNeighborsClassifier(metric= 'minkowski', n_neighbors = 3, p= 1), GaussianNB()],
#                                                             'final_estimator': [LogisticRegression(solver= 'liblinear', C = 0.23357214690901212), 
#                                                              KNeighborsClassifier(metric= 'minkowski', n_neighbors = 3, p= 1), GaussianNB()],

In [68]:
lr = LogisticRegression()

In [71]:
stacking_classifier_optimal = RandomizedSearchCV(StackingClassifier(estimators[0]), 
                                                          {
                                                            'estimators': estimators,
                                                            'final_estimator': estimators[0],
                                                            'stack_method' : ['predict_proba', 'decision_function', 'predict', 'auto']
                                                          },
                                                          n_iter=4,
                                                          n_jobs=4,
                                                          scoring='accuracy',
                                                          random_state=71
                                                          ).fit(X_train, y_train)
stacking_classifier_optimal.best_params_

ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Machine_Learning\ML\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Machine_Learning\ML\venv\lib\site-packages\sklearn\ensemble\_stacking.py", line 660, in fit
    return super().fit(X, y_encoded, sample_weight)
  File "d:\Machine_Learning\ML\venv\lib\site-packages\sklearn\ensemble\_stacking.py", line 190, in fit
    self._validate_params()
  File "d:\Machine_Learning\ML\venv\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "d:\Machine_Learning\ML\venv\lib\site-packa